In [1]:
from hierarchical_model import *
import numpy as np
import xml.etree.ElementTree as ET
from sklearn.cluster import KMeans
from sklearn.preprocessing import MaxAbsScaler
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.wsd import lesk
import pickle
import os

In [2]:
train_documents = read_train_data('access', 'n')
vectorizer = vectorize(train_documents)
vectors = transform(vectorizer, train_documents)

In [4]:
vectors

<1000x8246 sparse matrix of type '<type 'numpy.float64'>'
	with 29925 stored elements in Compressed Sparse Row format>

In [8]:
# scale the feature vectors
scaler = MaxAbsScaler(copy=False)
vectors = scaler.fit_transform(vectors)

In [9]:
vectors

<1000x8246 sparse matrix of type '<type 'numpy.float64'>'
	with 29925 stored elements in Compressed Sparse Row format>

In [12]:
hr = AgglomerativeClustering(n_clusters = get_num_meanings('access', 'n'), affinity = 'cosine', linkage = 'complete')


In [15]:
hr.fit(vectors.toarray())

AgglomerativeClustering(affinity='cosine', compute_full_tree='auto',
            connectivity=None, linkage='complete',
            memory=Memory(cachedir=None), n_clusters=5, n_components=None,
            pooling_func=<function mean at 0x10a19a5f0>)

In [16]:
hr.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 4, 4, 0, 4, 0, 0, 0, 0, 1, 0, 3, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 4, 4, 4, 0, 1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 4, 0, 0, 0, 0, 0, 0, 0, 4, 3, 0, 0, 4, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 4,
       0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 4, 1, 0, 0, 4, 0, 0, 4, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 4, 0, 1, 0, 4, 0, 0, 0, 0, 0, 0, 1, 4, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 4, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 4, 0, 4, 0, 0,
       0, 0,

In [18]:
hr.labels_.shape

(1000,)

In [20]:
np.bincount(hr.labels_)

array([857,  54,   3,   7,  79])

### See if this method of clustering gives stable results

In [22]:
for _ in xrange(4):
    train_documents = read_train_data('access', 'n')
    vectorizer = vectorize(train_documents)
    vectors = transform(vectorizer, train_documents)
    hr = AgglomerativeClustering(n_clusters = get_num_meanings('access', 'n'), affinity = 'cosine', linkage = 'complete')
    hr.fit(vectors.toarray())
    print np.bincount(hr.labels_)

[ 77 857   8  39  19]
[ 77 857   8  39  19]
[ 77 857   8  39  19]
[ 77 857   8  39  19]


### Predict new text

In [23]:
sent = '$ 10 milion redesign on all guestrooms in 2002. King/Doubles , coffee maker , in-room safe suitable for laptops , workingdesk with data port , newspaper , high speed , wireless internet access .'

In [26]:
vectorizer = vectorize([sent])

In [32]:
vector = transform(vectorizer, [sent])

In [34]:
hr.fit_predict(vector.toarray())

ValueError: Found array with 1 sample(s) (shape=(1, 8246)) while a minimum of 2 is required by AgglomerativeClustering.